In [1]:
import pandas as pd
import numpy as np

In [2]:
class DisjointSetUnion:
    
    def __init__(self):
        self.parent = {}
        self.cluster_count = 0
        self.biomass = {}
        self.initial_value = {}
        self.partial_join = []
    
    def add(self, x, val):
        if x in self.parent:
            return
        self.parent[x] = x
        self.cluster_count += 1
        self.biomass[x] = val
        self.initial_value[x] = val
    
    def find(self, x):
        if x != self.parent[x]:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]
    
    def union(self, x, y):
        xp, yp = self.find(x), self.find(y)
        if xp == yp:
            return False
        self.parent[xp] = yp
        self.biomass[yp] += self.biomass[xp]
        self.cluster_count -= 1
        return True
    
    def partial_union(self, x, y,distance_matrix,distance_matrix_array):
        capacity = 20000 - self.biomass[y]
        biomass_per_dist = []
        for grid in range(2418):
            if self.find(grid) == x:
                depo = y
                weight = 1e9
                if distance_matrix_array[grid][depo] != 0:
                    weight = self.initial_value[grid] / distance_matrix_array[grid][depo]
                value = self.initial_value[grid]
                self.parent[grid] = grid
                self.biomass[grid] = self.initial_value[grid]
                biomass_per_dist.append([weight, value, grid])
        biomass_per_dist.sort(key=lambda l: l[0], reverse=True)
        remaining=[]
        for index, node in enumerate(biomass_per_dist):
            value = node[1]
            grid = node[2]
            if 0.001 <= capacity and value <= capacity:
                self.biomass[y] += value
                self.parent[grid] = y
                capacity -= value
            elif 0.001 <= capacity:
                self.biomass[y] += capacity
                self.biomass[grid] -= capacity
                self.initial_value[grid] -= capacity
                biomass_per_dist[index][1] -= capacity
                remaining=biomass_per_dist[index:]
                self.partial_join.append([capacity, grid, depo])
                capacity = 0
                break
            else:
                break
        if len(remaining)==0:
            mask = []
            value = []
            for node in remaining:
                mask.append(node[2])
                value.append(node[1])
            temp_matrix = distance_matrix.iloc[mask, mask].T.values
            temp_cost = np.dot(temp_matrix, value)
            temp_index = np.argmin(temp_cost)
            parent = mask[temp_index]
            self.parent[parent] = parent
            self.biomass[parent] = self.initial_value[parent]
            for grid in mask:
                if grid != parent:
                    self.parent[grid] = parent
                    self.biomass[parent] += self.initial_value[grid]
    
    def is_connected(self, x, y):
        if x in self.parent and y in self.parent:
            return self.find(x) == self.find(y)
        else:
            return False

In [3]:
class Knapsack_Model:
    
    def __init__(self,distance_matrix):
        self.distance_matrix=distance_matrix
        self.distance_matrix_array=distance_matrix.values.copy()
        self.error_margin=0.001

    def knapsack_mask(self, biomass_value, depo_locations):
        biomass_per_dist = []
        residue = [0] * 2418
        for grid in range(2418):
            for depo in depo_locations:
                wt = 1e9 if self.distance_matrix_array[grid][depo] < self.error_margin else biomass_value[grid] / self.distance_matrix_array[grid][depo]
                biomass_per_dist.append([wt, grid, depo])
                residue[grid] = biomass_value[grid]
        biomass_per_dist.sort(key=lambda l: l[0], reverse=True)
        knapsack = {depo: [0] * 2418 for depo in depo_locations}
        utility = {depo: 0 for depo in depo_locations}
        index = 0
        while index < len(biomass_per_dist):
            node = biomass_per_dist[index]
            grid, depo = node[1], node[2]
            if residue[grid] <= 0.001 or utility[depo] + 0.001 >= 20000:
                index += 1
                continue
            quantity = min(residue[grid], 20000-utility[depo])
            knapsack[depo][grid] = quantity
            utility[depo] += quantity
            residue[grid] -= quantity
            index += 1
        return knapsack, utility
    
    def location_mask_optimizer(self,biomass_value,depo_locations, refinery_mask):
        depo_locations_new = []
        refinery_mask_new = [[] for i in refinery_mask]

        #knapsack creation
        knapsack, depo_utility = self.knapsack_mask(biomass_value, depo_locations)
        
        # Calculate Refinery Locations
        refinery_locations = self.calculate_refinery_locations(refinery_mask, depo_utility)

        depo_to_refinery = {}
        for index, mask in enumerate(refinery_mask):
            for i in mask:
                depo_to_refinery[i] = index
        cluster_mask = []
        value_mask = []
        for index, depo in enumerate(depo_locations):
            mask = knapsack[depo]
            temp_cluster_mask = []
            temp_value_mask = []
            for node, value in enumerate(mask):
                if value > self.error_margin:
                    temp_value_mask.append(value)
                    temp_cluster_mask.append(node)
            cluster_mask.append(temp_cluster_mask)
            value_mask.append(temp_value_mask)
        #Calculating New Depo and Refinary mask
        minicost = 0
        for index, mask in enumerate(cluster_mask):
            depo_id = depo_locations[index]
            ref = refinery_locations[depo_to_refinery[depo_id]]
            temp_matrix = self.distance_matrix.iloc[mask, mask].T.values
            temp_value = value_mask[index]
            temp_cost = np.dot(temp_matrix, temp_value)
            for idx, i in enumerate(mask):
                temp_cost[idx] += depo_utility[depo_id] * self.distance_matrix_array[i][ref]
            temp_index = np.argmin(temp_cost)
            minicost += np.min(temp_cost)
            depo_locations_new.append(mask[temp_index])

        for index, depo in enumerate(depo_locations):
            refinery_index = depo_to_refinery[depo]
            depo_new = depo_locations_new[index]
            refinery_mask_new[refinery_index].append(depo_new)

        return depo_locations_new, refinery_mask_new, minicost
    
    def calculate_refinery_locations(self,refinery_mask,depo_utility):
        refinery_locations = []
        for mask in refinery_mask:
            temp_matrix = self.distance_matrix.iloc[mask, :].T.values
            temp_value = [depo_utility[depo] for depo in mask]
            temp_cost = np.dot(temp_matrix, temp_value)
            temp_index = np.argmin(temp_cost)
            refinery_locations.append(temp_index)
        return refinery_locations
    
    def train(self,biomass_value,depo_locations, refinery_mask):
        depo_locations_new=[]
        refinery_mask_new=[]
        minicost=-1
        for epoch in range(10):
            depo_locations_new, refinery_mask_new, minicost = self.location_mask_optimizer(biomass_value,depo_locations, refinery_mask)
            if np.array_equal(depo_locations_new, depo_locations):
                break
            depo_locations=depo_locations_new
            refinery_mask=refinery_mask_new
        return depo_locations_new, refinery_mask_new, minicost

    def predict(self,biomass_value,depo_locations,refinery_mask):
        #knapsack creation
        knapsack, depo_utility = self.knapsack_mask(biomass_value, depo_locations)
        
        # Calculate Refinery Locations
        refinery_locations = self.calculate_refinery_locations(refinery_mask, depo_utility)
        return refinery_locations,knapsack


In [4]:
def truncate_float(f, n):
        '''Truncates/pads a float f to n decimal places without rounding'''
        str_value = '%.12f' % f
        integer_part, _, decimal_part = str_value.partition('.')
        return float('.'.join([integer_part, (decimal_part + '0' * n)[:n]]))

In [5]:
class BiomassSupplyChain:
    
    def __init__(self, distance_file, biomass_history_file, sample_submission_file, biomass_forecast_file):
        self.distance_matrix = pd.read_csv(distance_file, index_col=0)
        self.biomass_history = pd.read_csv(biomass_history_file)
        self.sample_submission = pd.read_csv(sample_submission_file)
        self.biomass_forecast = pd.read_csv(biomass_forecast_file,index_col=0)
        self.error_margin=0.001
        if (max(self.biomass_forecast.sum()['2018':'2019'])<380000):
            self.num_depots = 15
            self.num_refineries = 3
        else :
            self.num_depots = 20
            self.num_refineries = 4
        print("Number of depose needed :",self.num_depots)
        print("Number of refineries needed :",self.num_refineries)
        # apply average startegy
        self.biomass_value = (self.biomass_forecast['2018'].values + self.biomass_forecast['2019'].values) / 2
        self.distance_matrix_array = self.distance_matrix.values.copy()
        self.cluster_size = []


    def preprocess_data(self):
        # Drop unnecessary columns
        self.biomass_forecast.drop(columns=['Latitude', 'Longitude'], inplace=True)

    def calculate_weights_matrix(self):
        distance_matrix_copy = self.distance_matrix.values.copy()
        weights = self.biomass_value[:, np.newaxis] + self.biomass_value[np.newaxis, :]
        for i in range(self.distance_matrix.shape[0]):
            weights[i][i] = 0
            distance_matrix_copy[i][i] = 1
        weights_matrix = weights / distance_matrix_copy
        weights_matrix[weights_matrix == np.inf] = 1e9
        return weights_matrix
    
    def minimum_spanning_tree_clustering(self,weights_matrix):
        # Initialize Disjoint Set Union
        edges = []
        for i in range(2418):
            for j in range(2418):
                if i != j:
                    edges.append([weights_matrix[i][j], i, j])
        edges.sort( key=lambda l: l[0], reverse=True)
        mst = DisjointSetUnion()
        for i in range(2418):
            mst.add(i, self.biomass_value[i])
        # Creating Minimum Spanning Tree
        for edge in edges:
            i = edge[1]
            j = edge[2]
            if (not mst.is_connected(i,j)) and mst.biomass[i] < mst.biomass[j] and mst.find(i) == i and mst.find(j) == j and mst.biomass[j] + self.error_margin < 20000:
                if mst.biomass[j] + mst.biomass[i] +self.error_margin <= 20000:
                    mst.union(i, j)
                else:
                    mst.partial_union(i, j,self.distance_matrix,self.distance_matrix_array)
        # Select clusters and make mask
        unique_clusters_id=np.unique(list(mst.parent.values()))
        cluster_mask={}
        clusters=[]
        for cluster_id in unique_clusters_id:
            clusters.append([cluster_id,mst.biomass[cluster_id]])
            cluster_mask[cluster_id]=[]
        for grid in range(2418):
            cluster_mask[mst.find(grid)].append(grid)
        clusters.sort(key=lambda l:l[1],reverse=True)
        selected_clusters=clusters[:self.num_depots]
        # Calculate Initial Depo Locations 
        depo_loc=self.calculate_initial_depot_locations(selected_clusters,cluster_mask,mst.partial_join)
        return depo_loc

    def calculate_initial_depot_locations(self,selected_clusters,cluster_mask,partial_joins):
        depo_loc = []
        for selected_cluster in selected_clusters:
            cluster_id=selected_cluster[0]
            mask=cluster_mask[cluster_id]
            temp_matrix = self.distance_matrix.iloc[mask, mask].T.values
            temp_value = self.biomass_value[mask]
            temp_cost = np.dot(temp_matrix, temp_value)
            partial_edge = []
            for join in partial_joins:
                if join[2] == cluster_id:
                    partial_edge.append([join[0], join[1]])
            for index, i in enumerate(mask):
                cost = 0
                for j in partial_edge:
                    cost += (j[0] * self.distance_matrix.values[j[1]][i])
                temp_cost[index] += cost
            temp_index = np.argmin(temp_cost)
            depo_loc.append(mask[temp_index])
        return depo_loc
    
    def create_refinery_mask(self,model,index, depo_locations, refinery_mask, depo_locations_new, refinery_mask_new, minicost):
        size1 = len(refinery_mask[0])
        size2 = len(refinery_mask[1])
        size3 = len(refinery_mask[2])
        if size1 == 5 and size2 == 5 and size3 == 5:
            # Base case
            temp_depo_locations, temp_refinery_mask, temp_minicost = model.train(self.biomass_value,depo_locations, refinery_mask)
            if temp_minicost < minicost[0]:
                depo_locations_new[:] = temp_depo_locations
                refinery_mask_new[:] = temp_refinery_mask
                minicost[0] = temp_minicost
                # print(minicost)
            return
        # First
        if size1 < 5 and (size1 >= size2 or refinery_mask[1][size1] > depo_locations[index]) and \
                (size1 >= size3 or refinery_mask[2][size1] > depo_locations[index]):
            refinery_mask[0].append(depo_locations[index])
            self.create_refinery_mask(model,index + 1, depo_locations, refinery_mask, depo_locations_new, refinery_mask_new, minicost)
            refinery_mask[0].pop()  # Back propagation
        # Second
        if size2 < 5 and (size2 >= size1 or refinery_mask[0][size2] < depo_locations[index]) and \
                (size2 >= size3 or refinery_mask[2][size2] > depo_locations[index]):
            refinery_mask[1].append(depo_locations[index])
            self.create_refinery_mask(model,index + 1, depo_locations, refinery_mask, depo_locations_new, refinery_mask_new, minicost)
            refinery_mask[1].pop()  # Back propagation
        # Third
        if size3 < 5 and (size3 >= size1 or refinery_mask[0][size3] < depo_locations[index]) and \
                (size3 >= size2 or refinery_mask[1][size3] < depo_locations[index]):
            refinery_mask[2].append(depo_locations[index])
            self.create_refinery_mask(model,index + 1, depo_locations, refinery_mask, depo_locations_new, refinery_mask_new, minicost)
            refinery_mask[2].pop()  # Back propagation


    def knapsack_optimization(self,depo_locations):
        model=Knapsack_Model(self.distance_matrix)
        refinery_mask=[[] for _ in range(self.num_refineries)]
        depo_locations_new=[]
        refinery_mask_new=[]
        minicost=[3*1e8] # 20000(depo capacity) *1000(max distance)*15(no of depo)
        depo_locations=np.sort(depo_locations) # for greedy approch
        self.create_refinery_mask(model,0,depo_locations,refinery_mask,depo_locations_new,refinery_mask_new,minicost)
        refinery_locations,_ = model.predict(self.biomass_value,depo_locations_new,refinery_mask_new)
        return depo_locations_new,refinery_locations,refinery_mask_new,minicost[0]
    
    def get_knapsack_mask(self,depo_locations):
        model=Knapsack_Model(self.distance_matrix)
        knapsack_mask_2018,_=model.knapsack_mask(self.biomass_forecast['2018'].values,depo_locations)
        knapsack_mask_2019,_=model.knapsack_mask(self.biomass_forecast['2019'].values,depo_locations)
        return knapsack_mask_2018,knapsack_mask_2019
    
    def get_output(self,depo_locations,refinery_locations,refinery_mask,knapsack_mask_2018,knapsack_mask_2019):
        #Locations
        locations=[]
        for depo in depo_locations:
            locations.append([20182019,'depot_location',depo,np.nan,np.nan])
        for refinery in refinery_locations:
            locations.append([20182019,'refinery_location',refinery,np.nan,np.nan])
        location_df=pd.DataFrame(locations,columns=self.sample_submission.columns)

        #Forecast
        biomass_forecast=self.sample_submission[self.sample_submission['data_type']=='biomass_forecast']
        forecast=np.concatenate((self.biomass_forecast['2018'],self.biomass_forecast['2019']))
        biomass_forecast['value']=forecast
        biomass_forecast=biomass_forecast.reset_index().drop(columns=['index'])

        #Transport
        transport=[]
        ##Depo
        depovalue=[{},{}]
        for depo in depo_locations:
            mask=knapsack_mask_2018[depo]
            cost=0
            for grid,value in enumerate(mask):
                if(value>0):
                    value=truncate_float(value,3)
                    transport.append([2018,'biomass_demand_supply',grid,depo,value])
                    cost+=value
            depovalue[0][depo]=cost
        for depo in depo_locations:
            mask=knapsack_mask_2019[depo]
            cost=0
            for grid,value in enumerate(mask):
                if(value>0):
                    value=truncate_float(value,3)
                    transport.append([2019,'biomass_demand_supply',grid,depo,value])
                    cost+=value
            depovalue[1][depo]=cost
        ##Refinery
        for index,mask in enumerate(refinery_mask):
            for depo in mask:
                value=depovalue[0][depo]
                source=depo
                destination=refinery_locations[index]
                transport.append([2018,'pellet_demand_supply',source,destination,value])
        for index,mask in enumerate(refinery_mask):
            for depo in mask:
                value=depovalue[1][depo]
                source=depo
                destination=refinery_locations[index]
                transport.append([2019,'pellet_demand_supply',source,destination,value])
        transport_df=pd.DataFrame(transport,columns=self.sample_submission.columns)

        #submission
        submission=pd.concat([location_df,biomass_forecast,transport_df],axis=0)
        path='./Biomass Results/'
        submission.to_csv(path,'submission.csv',index=False)


In [6]:
# Instantiate BiomassSupplyChain class
supply_chain = BiomassSupplyChain(
    distance_file=r"./Dataset/Distance_Matrix.csv",
    biomass_history_file=r"./Dataset/Biomass_History.csv",
    sample_submission_file=r"./Dataset/sample_submission.csv",
    biomass_forecast_file=r"./Biomass Results/Biomass_Forecast.csv"
)
# Preprocess data
supply_chain.preprocess_data()

# Calculate weights matrix
weights_matrix = supply_chain.calculate_weights_matrix()

# Calculate depo locations
initial_depo_loc=supply_chain.minimum_spanning_tree_clustering(weights_matrix)

# Calculate refinary locations and new depo location using knapsack optimization
depo_locations,refinery_locations,refinery_mask,minicost=supply_chain.knapsack_optimization(initial_depo_loc)
print("Cost of transportation :",minicost*0.001)

# Transport Network for 2018 and 2019
knapsack_mask_2018,knapsack_mask_2019=supply_chain.get_knapsack_mask(depo_locations)

# Generate test file
supply_chain.get_output(depo_locations,refinery_locations,refinery_mask,knapsack_mask_2018,knapsack_mask_2019)

Number of depose needed : 15
Number of refineries needed : 3


/tmp/ipykernel_18/1921851467.py:33: RuntimeWarning: divide by zero encountered in divide
  weights_matrix = weights / distance_matrix_copy


Cost of transportation : 33355.665314100406


/tmp/ipykernel_18/1921851467.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  biomass_forecast['value']=forecast
